In [1]:
import struct
from datetime import datetime

In [2]:
with open('net.cap', 'rb') as f:
    data = f.read()

In [3]:
raw = data[:]
pcap_file_header = raw[:24]
raw = raw[24:]

ethernet_packets = []
timestamps = []
while raw:
    packet_header = raw[:16]
    raw = raw[16:]
#     print(list(packet_header))
    tsec = struct.unpack('I', packet_header[:4])[0]
    tmsec = struct.unpack('I', packet_header[4:8])[0]
    timestamps.append(datetime.utcfromtimestamp(tsec + tmsec/1_000_000))
    length = struct.unpack('I', packet_header[-4:])[0]
    packet = raw[:length]
    ethernet_packets.append(packet)
    raw = raw[length:]

In [4]:
assert sorted(timestamps) == timestamps

In [5]:
ip_packets = []
for packet in ethernet_packets:
#     print(len(packet))
    mac = packet[:6]
    packet = packet[6:]
    mac = ':'.join(hex(n)[2:] for n in struct.unpack(6 * 'B', mac))
#     print('destination mac', mac)
    mac = packet[:6]
    packet = packet[6:]
    mac = ':'.join(hex(n)[2:] for n in struct.unpack(6 * 'B', mac))
#     print('origin mac', mac)
    type_length = packet[:2]
    packet = packet[2:]
    type_length = struct.unpack('BB', type_length)
#     print(type_length)
#     print(len(packet))
    ip_packets.append(packet)

In [6]:
tcp_packets = []

for packet in ip_packets:
#     print(packet)
    bits = format(packet[0], '#010b')
#     print(bits)
    ihl = int(bits[-4:], base=2)
    version = int(bits[:-4], base=2)
    print('version', version, 'IHL', ihl)
    assert ihl == 5
    
    bits = format(packet[1], '#010b')
#     print(bits)
    ecn = int(bits[-2:], base=2)
    dscp = int(bits[:-2], base=2)
    print('dscp', dscp, 'ecn', ecn)
    
    tl = struct.unpack('>H', packet[2:4])[0]
    print('total length', tl, 'len(packet)', len(packet))

    id_ = struct.unpack('>H', packet[4:6])[0]
    print('id', id_)
    
    print('flags + fragment offset', format(struct.unpack('>H', packet[6:8])[0], '#018b'))
    
    print('ttl', format(struct.unpack('B', packet[8:9])[0]))
    
    protocol = struct.unpack('B', packet[9:10])[0]
    assert protocol == 6
    print('protocol (TCP = 6)', protocol)
    
    print('header checksum', format(struct.unpack('>H', packet[10:12])[0]))

    print('source ip', struct.unpack('BBBB', packet[12:16]))
    print('destination ip', struct.unpack('BBBB', packet[16:20]))

    tcp_packets.append(packet[20:])
    print('packet size', len(packet[20:]))

version 4 IHL 5
dscp 0 ecn 0
total length 64 len(packet) 64
id 53251
flags + fragment offset 0b0000000000000000
ttl 64
protocol (TCP = 6) 6
header checksum 11502
source ip (192, 168, 0, 101)
destination ip (192, 30, 252, 154)
packet size 44
version 4 IHL 5
dscp 18 ecn 0
total length 60 len(packet) 62
id 0
flags + fragment offset 0b0100000000000000
ttl 41
protocol (TCP = 6) 6
header checksum 54189
source ip (192, 30, 252, 154)
destination ip (192, 168, 0, 101)
packet size 42
version 4 IHL 5
dscp 0 ecn 0
total length 64 len(packet) 64
id 56901
flags + fragment offset 0b0000000000000000
ttl 64
protocol (TCP = 6) 6
header checksum 7852
source ip (192, 168, 0, 101)
destination ip (192, 30, 252, 154)
packet size 44
version 4 IHL 5
dscp 18 ecn 0
total length 60 len(packet) 62
id 0
flags + fragment offset 0b0100000000000000
ttl 41
protocol (TCP = 6) 6
header checksum 54189
source ip (192, 30, 252, 154)
destination ip (192, 168, 0, 101)
packet size 42
version 4 IHL 5
dscp 0 ecn 0
total length 6

version 4 IHL 5
dscp 18 ecn 0
total length 1476 len(packet) 1476
id 2788
flags + fragment offset 0b0100000000000000
ttl 41
protocol (TCP = 6) 6
header checksum 49985
source ip (192, 30, 252, 154)
destination ip (192, 168, 0, 101)
packet size 1456
version 4 IHL 5
dscp 0 ecn 0
total length 72 len(packet) 72
id 1855
flags + fragment offset 0b0000000000000000
ttl 64
protocol (TCP = 6) 6
header checksum 62890
source ip (192, 168, 0, 101)
destination ip (192, 30, 252, 154)
packet size 52
version 4 IHL 5
dscp 18 ecn 0
total length 1476 len(packet) 1476
id 2789
flags + fragment offset 0b0100000000000000
ttl 41
protocol (TCP = 6) 6
header checksum 49984
source ip (192, 30, 252, 154)
destination ip (192, 168, 0, 101)
packet size 1456
version 4 IHL 5
dscp 0 ecn 0
total length 72 len(packet) 72
id 27170
flags + fragment offset 0b0000000000000000
ttl 64
protocol (TCP = 6) 6
header checksum 37575
source ip (192, 168, 0, 101)
destination ip (192, 30, 252, 154)
packet size 52
version 4 IHL 5
dscp 18 e

In [7]:
for packet in tcp_packets:
    print('source port', struct.unpack('>H', packet[0:2]))
    print('dest port', struct.unpack('>H', packet[2:4]))
    print('sequence number', struct.unpack('>I', packet[4:8]))
    print('ack number', struct.unpack('>I', packet[8:12]))
    bits = format(struct.unpack('B', packet[12:13])[0], '#010b')
    offset = int(bits[2:6], base=2)
#     print('data offset', offset)
    print('flags', format(struct.unpack('B', packet[13:14])[0], '#010b'))
#     print('window size', struct.unpack('>H', packet[14:16])[0])
#     print('checksum', struct.unpack('>H', packet[16:18])[0])
#     print('urgent pointer', struct.unpack('>H', packet[18:20])[0])
    header_length = offset * 4
    print(packet[header_length:])
    print()

source port (59295,)
dest port (80,)
sequence number (1588273765,)
ack number (0,)
flags 0b00000010
b''

source port (80,)
dest port (59295,)
sequence number (150766358,)
ack number (1588273766,)
flags 0b00010010
b'\xb2\xf8'

source port (59295,)
dest port (80,)
sequence number (1588273765,)
ack number (0,)
flags 0b00000010
b''

source port (80,)
dest port (59295,)
sequence number (150766358,)
ack number (1588273766,)
flags 0b00010010
b'\x80\xe6'

source port (59295,)
dest port (80,)
sequence number (1588273765,)
ack number (0,)
flags 0b00000010
b''

source port (80,)
dest port (59295,)
sequence number (150766358,)
ack number (1588273766,)
flags 0b00010010
b'\xc9\xa0'

source port (59295,)
dest port (80,)
sequence number (1588273765,)
ack number (0,)
flags 0b00000010
b''

source port (80,)
dest port (59295,)
sequence number (150766358,)
ack number (1588273766,)
flags 0b00010010
b'\x1b\xb6'

source port (59295,)
dest port (80,)
sequence number (1588273765,)
ack number (0,)
flags 0b00000

source port (59295,)
dest port (80,)
sequence number (1588273846,)
ack number (150776327,)
flags 0b00010000
b''

source port (80,)
dest port (59295,)
sequence number (150793415,)
ack number (1588273846,)
flags 0b00010000
b'\xc0\xd0\xcai\xc6\xaeo+J[SRW\xd7\xac\x9e\xb6K\xb6\xef\xc8\xf40\xf9\\\xf1\xf4\xa5V\x17\xe5\x8a\xf8\xb6N\xdf\xcb\xa3\xbe\xbdv>\xf9\xfd\x95\xfe:x\xcb\xe2/\x84\xe5\xf0\x07\xc5\x9b+\x17\xf1v\xb5\xa4\xdcn\xbe\xd3l\xe4\x83L\x9df-\xb1\xbe\xce\xd9X\xc8\xdc\xaa\xc5]\x81 \x13\x8e\x95\xf6^\x81\xe2\x984k\xbd>\xc7T\xb6\x8e{\xcb;_\xec"\xd8kX\xa7\x86GM\x92+r8p\x17\xe6\xc8\xdd\xdf\x04W\x83^\xa57W\xdbR\xf8\x14\x9a_7ti\x86\xc3U\xc3T\xa9\x82\x9b\xb3\xb72\xf9\xab>\xc7\x98\xf8\xbf\xc3z>\xa5\xe2\xddW\\\xb1\x8bJ\xd5\xae\xc3]xc\xc4\x9aq_&\xea\xd4\xb2\x82\x88\xe3\x00\xee\x11\xc8\\J\x9c6:\xf1P\xf8\x9e\xfe\x03\x0f\x83\x06\x9c\x1e\x19\xfc;\xf6\x0b\xfdR\x17\x8d\x99u\xbb;\xabY-`\xb8\x1d\xcc\x91\xect}\xdc\x822z\xe6\xbd\nXxF|\xf1[\xff\x00\xc0\xe8cW\x17R\xa5(\xd3\xa8\xfe\x05k>\xcdY4\xfei\xfc\x8f\x80?